<a href="https://colab.research.google.com/github/otluiz/Covid-19_kaggle/blob/master/Copy_of_Covid_19_Kaggle_Textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import zipfile
import pandas as pd
import glob
import json
import seaborn as sns
import spacy
import nltk
from IPython.core.display import HTML
from matplotlib import pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#!pip install -U spacy
#!pip install scispacy
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz
#!python -m spacy download en
import en_core_sci_md
import scispacy
import zipfile

Chama os drives da Google e monta um novo diretório neste 'notebook'. Precisa de uma conexão (URL) onde a Google gera uma 'auth' para entrar no meu Google Drive

First, you'll need to mount your Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Conecta ao diretório do meu Google Drive onde estão os dados da Covid-19 que está compactado.
Será descompactado (unzip) no meu Google Drive


In [ ]:
path = '/content/gdrive/My Drive/CORD-19-research-challenge.zip'
from zipfile import *
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

Criação do DataFrame com os textos dos artigos


DataFrame no formato JSON

In [ ]:
features_corona = {'paper_id': [],
                   'title' : [],
                   'abstract' : [],
                   'text' : []}

Testando o tipo "features_corona"


In [ ]:
type (features_corona)

Transformar o tipo dict em dataframe


In [ ]:
corona_df = pd.DataFrame.from_dict(features_corona)

Verificando o novo tipo

In [ ]:
type (corona_df)

Pandas é o "default for Machine Learning in Python projects"


In [ ]:
corona_df.head()

Ler os arquivos JSON a partir da raiz '/' para popular o futuro dataframe (corona_df) a ser criado mais adiante


In [ ]:
json_filenames = glob.glob(f'{"./"}//**/*.json', recursive = True)
print(json_filenames)

In [ ]:
len(json_filenames)

Cada arquivo JSON seráá uma linha do dataframe, populando assim o corona_df


In [ ]:
def retorna_corona_df(json_filenames, df):
  for file_name in json_filenames:
    linha_dados = {'paper_id': None, 'title': None, 'abstract': None, 'text': None}

    with open(file_name) as json_data:
      if file_name == './sample_data/anscombe.json': # ignora os anscombe.json
        continue
      #if file_name == './MeuMestrado/ProjetoGeolocalização/node_modules/gulp/node_modules/tildify/*.json': 
      #  continue

      data = json.load(json_data)

      ## .strip() : ' 345 ' -> '345' retira os espaços
      linha_dados['paper_id'] = data['paper_id'].strip()
      linha_dados['title'] = data['metadata']['title'].strip()

      abstract_list = [abstract['text'] for abstract in data['abstract']]
      abstract = '\n'.join(abstract_list)
      linha_dados['abstract'] = abstract.strip()

      text_list = [text['text'] for text in data['body_text']]
      text = '\n'.join(text_list)
      linha_dados['text'] = text.strip()

      df = df.append(linha_dados, ignore_index = True)
  return df

In [ ]:
corona_df = retorna_corona_df(json_filenames, corona_df)

In [ ]:
corona_df2 = corona_df[['paper_id', 'title', 'abstract', 'text']] ## criando um back-up do DataFrame


In [ ]:
#corona_df.drop(corona_df.tail(29315).index,inplace=True) # remove as ultimas linhas

In [ ]:
#corona_df.drop(corona_df.head(2).index,inplace=True) # drop first 2 rows

In [ ]:
corona_df.shape

(29315, 4)

In [ ]:
corona_df2.shape

(29315, 4)

In [ ]:
corona_df.head()

In [ ]:
corona_df['title'][29314] ## Analise de algum texto escolhido ao acaso

In [ ]:
corona_df.tail()

In [ ]:
corona_df['text'][2]

In [ ]:
len(corona_df['text'][2]) ## Quantas palavras têm este texto [2]?

**PERPROCESSAMENTO DE TEXTOS DOS ARTIGOS**
<PROCURANDO POR DADOS FALTANTES>

In [ ]:
sns.heatmap(corona_df.isnull()); ## procurando dados faltantes pelo mapa de calor

In [ ]:
len(corona_df[corona_df['paper_id'] == '']) # procurando por espaços vazios

0

In [ ]:
len(corona_df[corona_df['title'] == '']) # procurando por espaços vazios nos tíítulos

3583

In [ ]:
len(corona_df[corona_df['abstract'] == '']) # procurando por espaços vazios nos abstracts

8051

In [ ]:
len(corona_df[corona_df['text'] == '']) # procurando por espaços vazios nos textos

In [ ]:
#corona_df = corona_df2  ## reafzendo tudo porra!!

In [ ]:
corona_df = corona_df[corona_df['title'] != ''] ## apagando os artigos sem tíítulos

In [ ]:
corona_df = corona_df[corona_df['abstract'] != ''] ## apagando os artigos sem abstract

In [ ]:
corona_df.shape ## o que sobrou após a limpeza??

(20343, 4)

**REMOVENDO ARTIGOS DUPLICADOS**

In [ ]:
corona_df.drop_duplicates(['abstract', 'text', 'title'], inplace=True)

In [ ]:
corona_df.shape  ## vendo quem mandou 2 ou mais vezes o mesmo artigo - ahaha

(20211, 4)

**BASE DE DADOS DE AMOSTRA**


In [ ]:
corona_df = corona_df.sample(n = 500, random_state=1)

In [ ]:
corona_df.shape

(500, 4)

In [ ]:
corona_df.head()

In [ ]:
corona_df.tail()

In [ ]:
## Visual Text analisys
sample_text = corona_df['text'][11015]
sample_text

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz

In [ ]:
#!pip uninstall en_core_web_md
#!pip uninstall en_core_sci_sm
#!pip uninstall en_core_sci_md
!python -m spacy download en_core_web_md
!python -m spacy download en_core_sci_md
!python -m spacy link en_core_web_md en --force
#!pip install scispacy --force
#!python -m spacy link en_core_sci_md en --force

**FUNÇÃO PARA PREPROCESSAMENTO**


In [ ]:
#nlp = spacy.load('en')
#nlp = spacy.load('en_core_sci_md')
#nlp = en_core_sci_md.load(disable=['tagger', 'parser', 'ner'])

#nlp.max_length = 2000000

In [ ]:
#print('scispacy:', scispacy.__version__, " - ", "spacy:", spacy.__version__)
#!stopwnltk = nltk.corpus.stopwords.words("english")

In [ ]:
print(spacy.lang.en.stop_words.STOP_WORDS)

In [ ]:
sample_text

In [ ]:
len(sample_text)

26204

In [ ]:

# def spacy_tokenizer(sentence):
#  sentence = sentence.lower() ## coloca tudo em minúsculo
#  list = []   ## nlp(sentence)   nlp(sentence)
#  list = [word.lemma_ for word in sentence if word.is_stop or
#                                              word.like_num or
#                                              word.is_punct or
#                                              word.is_space)]                                              ]
#  list = ' '.join([str(element) for element in list])
#  return list

In [ ]:
# from spacy.lang.en import English

# # Load English tokenizer, tagger, parser, NER and word vectors
# nlp = English()

# text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
# #fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
# #indeed the vaguest idea where the wood and river in question were."""

# #  "nlp" Object is used to create documents with linguistic annotations.
# my_doc = nlp(text)

# # Create list of word tokens
# token_list = []
# for token in my_doc:
#     token_list.append(token.text)

# from spacy.lang.en.stop_words import STOP_WORDS

# # Create list of word tokens after removing stopwords
# filtered_sentence =[] 

# for word in token_list:
#     lexeme = nlp.vocab[word]
#     if lexeme.is_stop == False:
#         filtered_sentence.append(word) 
# print(token_list)
# print(filtered_sentence) 

In [ ]:
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and 
#fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had 
#indeed the vaguest idea where the wood and river in question were."""

meu_doc = nlp(sample_text)

token_list = []
def sentence(textos):
  for token in meu_doc:
    token_list.append(token.text)
  return token_list


In [ ]:
## junta as palavras numa mesma linha
def join_setences(textos):
  token_list = ' '.join([token_list for token_list in textos])
  return token_list

In [ ]:
# Create list of word tokens after removing stopwords
#filtered_sentence =[] 

def remove_stopwors(text):
  


In [ ]:
result = join_setences(sentence(text))
result
#print(result2 )

In [ ]:
teste = sample_text
result = spacy_tokenizer(teste)
result

In [ ]:
corona_df['text'] = corona_df['text'].apply(spacy_tokenizer)

In [ ]:
print(sample_text)

In [ ]:
len(sample_text)

In [ ]:
print(corona_df['text'][11015])